# <center>Projet Algorithmique avancée</center>
# <center>Livrable 1</center>

## **Contexte initial :**

Afin de réduire la consommation d'énergie et les émissions de gaz à effet de serre, l'ADEME (Agence de l’Environnement et de la Maîtrise de l’Energie) a lancé un appel à manifestation d’intérêt pour promouvoir la réalisation de démonstrateurs et d’expérimentations de nouvelles solutions de mobilité pour les personnes et les marchandises adaptées à différents types de territoires.
Dans l'optique de répondre à cet appel, notre structure CesiCDP a décidé d’orienter son étude sur la gestion de tournées de livraison.
L'objectif de cette étude est de calculer, sur un réseau routier, des tournées de camions permettant de relier des sous-ensembles de villes entre elles, puis de revenir au point de départ.
Ces tournées doivent être organisées de tel sorte que la date de retour du dernier camion à la base soit la plus proche possible de la date de départ afin de satisfaire les attentes de l'ADEME.


## **Versions du projet :**

### Les versions :

Notre projet a pour but de mener à bien notre étude.
Le projet est composé de 2 versions :

   - La versions de base :    
       - Le choix du modèle et le code en Python capable de résoudre des instances de taille importante (plusieurs milliers de villes). Cette résolution est sensée donner la tournée la plus courte possible passant par chaque ville.    
       - Une étude statistique du comportement expérimental de l'algorithme

   - La version évoluée :    
       - La version de base avec la prise en compte d'une ou plusieurs contraintes 


### Les contraintes du projet :

   - Fenêtre de temps de livraison pour chaque objet    
       - Interdiction de livrer hors de la fenêtre    
       - Possibilité d'attendre sur place l'ouverture de la fenêtre temporelle

   - Un nombre k camions disponibles simultanément pour effectuer les livraisons. Le calcul de la tournée devra inclure l’affectation des objets (et donc des points de livraison) aux différents camions disponibles, et minimiser non plus le temps total, mais la date de retour du dernier camion à la base.    
       - Capacité des camions (2 ou 3 dimensions) et encombrement des objets   
       - Certains objets ne peuvent être livrés que par certains camions

   - Chaque objet a un point de collecte spécifique

   - Le temps de parcours d’une arête varie au cours du temps (ce qui revient à faire varier sa longueur), pour représenter la variation du trafic



## **Modélisation :**

Afin de modéliser notre cas d’étude, nous devons être capable de représenter les villes, les chemins entre les villes et le temps nécessaire pour aller d’une ville à une autre. 
Le graphe est donc l’outil le plus adapté dans notre cas, il est noté G et défini par le couple $G=(V, E)$.      
   - V  est un ensemble de sommets. Chaque sommet représente une ville et chaque ville est numérotée par un un entier naturels non nul ($\mathbb{N}$*).    
   - E  est un ensemble d’arrêtes. Une arrête correspond à une paire de sommets et indique donc qu’ils sont reliés. Dans notre cas, une arrête indique donc que deux villes sont relié par une route carrossable par un camion.
   - La pondération sur les arrêtes correspond au temps nécessaire pour aller d’une ville à un autre.

Afin de construire notre modèle, nous devons déterminer les 3 composantes suivantes :    
   - Les variables : elles représentent les composantes du modèle qui peuvent être modifiées pour créer des configurations différentes.    
   - La fonction objectif : cette fonction assigne une valeur à chaque configuration différente. Le terme “objectif” vient du fait que l’objectif est d’optimiser cette fonction. Dans notre cas, nous voulons la minimiser.
   - Les contraintes : elles représentent les limitations sur les variables.

### Les variables :

L’étape la plus importante dans la construction d’un modèle est le choix des variables que nous allons utiliser. En effet, un mauvais choix de variable peut rendre le problème impossible à résoudre. 
Nous avons donc un ensemble $C={1, …, n_c}$ de villes qui doivent obtenir une livraison de marchandise provenant du dépôt. Nous considérons qu'une ville correspond au dépôt $a$ qui appartient donc à l'ensemble $C$, noté mathématiquement par $a \in C$. 
Une demande de produit $d_i$ est associée à chaque ville $i$ appartenant à $C$.
Une flotte de véhicules $V={1, …, n_v }$ est disponible au dépôt et chaque véhicule possède la même capacité (flotte homogène) $Q$  tel que $Q≥max⁡(d_i ), ∀i ∈ N .$
Pour toutes villes $i$  et $j , ∀i,j ∈ C$ , nous connaissons le coût $c_{i,j}$ de transport direct entre $i$  et $j$. 
Pour trouver l’ordre de visite des villes, nous définissons la variable de décision $x{_i^v},_j$.    
$x{_i^v},_j$ vaut $1$ si le véhicule $v ∈ V$ visite la ville $j$ après la ville $i ,$ $ou$ $0$ sinon. 

### La fonction objectif :
Pour rappel, nous devons définir les trajets de livraisons de tel sorte à minimiser la date de retour du dernier camion au point de départ (dépôt).
En définissant $y_i$ comme étant la charge résiduelle du véhicule après avoir desservi la ville $i \in C$ , il nous est possible d’écrire la fonction objectif : 

$$
    Min(\sum_{v=1}^{n_v} \sum_{i=1}^{n_c} \sum_{j=1}^{n_c} c_i, _j x^v_i, _j)  
$$


### Les contraintes :

#### Voici la liste de toutes les contraintes qui s’appliquent sur notre problème :


Cette équation assure qu’on part une et une seule fois de chaque ville, avec un seul véhicule.

$$
    \sum_{v∈V}\sum_{j∈C}x_{i,j}^{v} = 1, ∀i ∈ C
$$

Cette équation assure que le véhicule qui arrive chez une ville est le même que celui qui part de cette ville.

$$
    \sum_{j∈C}x_{i,j}^{v}-\sum_{j∈C}x_{j,i}^{v} = 0,  ∀i ∈ C, v ∈ V
$$

Cette équation assure que chaque véhicule ne sort qu’une seule fois du dépôt.

$$
     \sum_{j∈C}x_{0,j}^{v} =  1, ∀v ∈ V
$$

Cette équation assure le retour unique au dépôt pour chaque véhicule (ou tournée).

$$
    \sum_{j∈C}x_{j,n+1}^{v} =  1, ∀v ∈ V
$$

Ces équation définissent les contraintes de capacité et d’intégrité.

$$
    x_{i,j}^{v} = 1 ⇒ y_i - d_j = y_i, ∀i, j ∈ C, v ∈ V
$$

$$
    x_{i,j}^{v} ∈ {0, 1} , ∀i, j ∈ C, v ∈ V
$$

Ces équations sont des fonctions de mesure qui permettent respectivement de
quantifier la solution selon la distance totale parcourue.

La fonction de coût euclidien de la solution $ X = (x_{i,j}^{v}), ∀i, j ∈ C, ∀v ∈ V$ est définie par :

$$
    coût(X)=\sum_{v=1}^{n_v} \sum_{i=1}^{n_c} \sum_{j=1}^{n_c} c_i, _j x^v_i, _j
$$

## **Compléxité :**

Pour étudier la complexité de notre problème, nous allons le comparer à un autre problème connu afin de prouver qu’il est NP-difficile mais pas NP-complet. 
Pour ce faire, nous allons le comparé au problème du voyageur de commerce.
Nous partons du principe que nous voulons prouver que notre problème est NP-complet. Pour prouver que le problème de notre projet est NP-complet, nous devons démontrer deux choses : d'abord, qu'il est dans la classe NP (c'est-à-dire qu'une solution proposée peut être vérifiée en temps polynomial), et ensuite, qu'il est NP-difficile (c'est-à-dire qu'il est au moins aussi difficile que n'importe quel autre problème NP).

### 1 Prouver que notre problème est dans la classe NP : 

Pour montrer que notre problème est dans la classe NP, nous devons fournir un certificat ou une preuve vérifiable en temps polynomial pour une solution proposée. Dans le cas de notre problème, un certificat serait n séries de listes, chacune correspondant à la route empruntée par chaque camion. n étant le nombre de camions ayant effectué les tournées.
Nous fournirions également une instance du problème comprenant un graphe représentant les emplacements à visiter, les capacités des véhicules et les demandes des villes.
Nous cherchons donc à savoir, à l’aide d’un algorithme, si le certificat donné est bel et bien la solution optimale à notre problème, c’est-à-dire, que la différence de temps entre le moment où tous les camions ont quitté le dépôt (en même temps) et le moment où le dernier camion revient est la plus petite. 
Pour vérifier cela, nous devons donc être en capacité de tester toutes les combinaisons possibles sur l’instance afin de savoir quelle est la solution optimale et donc savoir si le certificat correspond à la solution optimale.
Cependant, l’algorithme de recherche de toutes les combinaisons possibles possède une complexité asymptotique bien supérieure à une complexité asymptotique polynomiale.
Il est donc impossible de vérifier un certificat en un temps polynomial, il n’est donc pas NP.

### 2 Prouver que notre problème est dans NP-difficile :

Afin de prouver que notre problème est dans NP-difficile, nous devons faire une réduction polynomiale d’un problème connu comme NP-difficile vers notre problème.
Le problème que nous avons choisi pour effectuer cette réduction polynomiale est le problème du voyageur du commerce (TSP) puisqu’il s’apparente beaucoup à notre problème dans le cas où nous avons un camion pour effectuer la tournée complète de toutes les villes.
Pour réduire le TSP à notre problème, nous pouvons construire une instance de notre problème avec un seul camion et une contrainte de capacité du camion égale au nombre de villes qu’il doit livrer. Nous fixons la capacité de chaque ville à 1, c’est-à-dire qu’à chaque ville visitée, notre camion se vide d'une unité. De plus, La distance entre deux villes dans notre problème correspond à la distance entre ces mêmes villes dans le TSP.

#### Voici l’algorithme permettant de transformer une instance de TSP à notre problème :

    Instance_TSP = matrice du graphe pondéré de TSP

    Nombre_De_Camions = 1

    Capacite_Camions = Nombre_Villes_Dans_Instance_TSP

    Capacite_Villes = [] (tableau vide)

    Pour i allant de 0 à Nombre_Villes_Dans_Instance_TSP :

        Rajouter une valeur dans Capacite_Villes égale à 1


A l’aide de cet algorithme, nous somme capable de transformer une instance du TSP en une instance du problème de notre problème.
Si nous trouvons une solution avec l'instance de notre problème construite dans cette réduction, cela implique que le véhicule peut visiter toutes les villes exactement une fois, satisfaisant ainsi l'exigence du TSP. De plus, la distance totale parcourue par le véhicule dans notre problème sera identique à la longueur du parcours du TSP. Ainsi, résoudre l'instance de notre problème résout également l'instance du TSP. 
Puisque le TSP est NP-complet, donc NP-difficile et que nous avons démontré une réduction polynomiale du TSP vers notre problème, nous pouvons conclure que le problème de notre projet est au moins aussi difficile que le TSP et donc, par conséquent, NP-difficile.

Nous pouvons donc finalement conclure en disant que nous avons prouver que notre problème est NP-diffcile mais pas NP. Il n’est donc pas NP-complet. Cependant, le fait que notre problème soit NP-difficile signifie que notre problème est extrêmement difficile à résoudre de manière exacte, car il nécessite généralement une exploration exhaustive de toutes les possibilités, ce qui peut prendre un temps exponentiel ou factoriel en fonction de la taille de l'instance du problème. Nous devrons donc utiliser des algorithmes permettant de trouver des solutions à notre problème en se rapprochant autant que possible de l’optimum, tout en pouvant être résolu en un temps acceptable.
